In [77]:
import pandas as pd
import numpy as np
import plotly
import plotly.express as px

diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes = diabetes.drop('Gender', axis = 1)
dupple_colm = list(diabetes.columns)
mask=diabetes.duplicated()
diabetes_dupl = diabetes[mask]

temp_dupped = diabetes.drop_duplicates(subset = dupple_colm)
#display(diabetes_dupped)

low_information_cols = [] 
for col in temp_dupped.columns:
    #наибольшая относительная частота в признаке
    top_freq = temp_dupped[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = temp_dupped[col].nunique() / temp_dupped[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
information_diabetes = temp_dupped.drop(low_information_cols, axis=1)
#print(f'Результирующее число признаков: {information_diabetes.shape[1]}')

def is_nan(num):
    if num == 0:
        return np.nan
    else:
        return num

temp_dupped['Glucose']=temp_dupped['Glucose'].apply(is_nan)
temp_dupped['BloodPressure']=temp_dupped['BloodPressure'].apply(is_nan)
temp_dupped['SkinThickness']=temp_dupped['SkinThickness'].apply(is_nan)
temp_dupped['Insulin']=temp_dupped['Insulin'].apply(is_nan)
temp_dupped['BMI']=temp_dupped['BMI'].apply(is_nan)
        
combine_data = temp_dupped.copy()

#отбрасываем столбцы с числом пропусков более 30% (100-70)
n = combine_data.shape[0] #число строк в таблице
thresh = n*0.7
combine_data = combine_data.dropna(thresh=thresh, axis=1)

#отбрасываем строки с числом пропусков более 2 в строке
m = combine_data.shape[1] #число признаков после удаления столбцов
combine_data = combine_data.dropna(thresh=m-2, axis=0)

values = {
    'Glucose': combine_data['Glucose'].median(),
    'BloodPressure': combine_data['BloodPressure'].median(),
    'SkinThickness': combine_data['SkinThickness'].median(),
    'BMI': combine_data['BMI'].median()
}
#заполняем оставшиеся записи константами в соответствии со словарем values
combine_data = combine_data.fillna(values)

def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - left*iqr
    upper_bound = quartile_3 + right*iqr
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr_mod(combine_data, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

"""def outliers_z_score_mod(data, feature, log_scale=False, left = 3, right = 3):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, cleaned = outliers_z_score_mod(combine_data, 'SkinThickness', log_scale=True, left = 3.7, right = 3.7)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')"""

Число выбросов по методу Тьюки: 0
Результирующее число записей: 761


C:\Users\lepni\AppData\Local\Temp\ipykernel_10572\1129394692.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_dupped['Glucose']=temp_dupped['Glucose'].apply(is_nan)
C:\Users\lepni\AppData\Local\Temp\ipykernel_10572\1129394692.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_dupped['BloodPressure']=temp_dupped['BloodPressure'].apply(is_nan)
C:\Users\lepni\AppData\Local\Temp\ipykernel_10572\1129394692.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

"def outliers_z_score_mod(data, feature, log_scale=False, left = 3, right = 3):\n    if log_scale:\n        x = np.log(data[feature]+1)\n    else:\n        x = data[feature]\n    mu = x.mean()\n    sigma = x.std()\n    lower_bound = mu - left * sigma\n    upper_bound = mu + right * sigma\n    outliers = data[(x < lower_bound) | (x > upper_bound)]\n    cleaned = data[(x > lower_bound) & (x < upper_bound)]\n    return outliers, cleaned\noutliers, cleaned = outliers_z_score_mod(combine_data, 'SkinThickness', log_scale=True, left = 3.7, right = 3.7)\nprint(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')\nprint(f'Результирующее число записей: {cleaned.shape[0]}')"